In [2]:
# Basic imports
import scipy, pandas as pd, numpy as np
from matplotlib import pyplot as plt

# set the constants
CUTOFF = -6.67
DAYS = 3

In [3]:
# reshape, remove unnecessary columns and add statistical filter + threshold
def proc(frame, lower_threshold, year):
    processed = frame.copy()
    for name in list(frame):
        if name[:4] != "tmin" and name != "GEOID20":
            processed = processed.drop(name, axis=1)
    
        
    stats = processed.set_index('GEOID20')
    stats = stats.apply(pd.DataFrame.describe, axis=1)
    stats["lower"] = stats["mean"]-1.645*stats["std"]

    processed = processed.melt(id_vars='GEOID20', var_name='Date', value_name='Temp')
    
    # conversion to Python datetime objects
    processed['Date'] = processed['Date'].map(lambda x: x.lstrip('tmin')) + str(year)
    processed['Date'] = pd.to_datetime(processed['Date'], format='%b%d%Y')

    # Set + sort by multi-level index using 'GEOID20' and 'Date'
    processed.set_index(['GEOID20', 'Date'], inplace=True)
    processed.sort_index(inplace=True)

    thresholds = stats.copy()
    thresholds.drop(columns=['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'], inplace=True)
    
    # for lower here we currently have this as 4.44
    thresholds['static']= lower_threshold
    
    merged = processed.reset_index().merge(thresholds.reset_index(), on='GEOID20', how='left').set_index(['GEOID20', 'Date'])

    return merged

In [4]:
# function to create a (not indexed) boolean series which tells you whether an entry is part of a consecutive series of length n
def consec_count_n(series, n):
    temp = []
    c = 0
    for i in range(len(series.values)):
        if series.values[i] == False:
            c = 0
        else:
            c += 1
        temp.append(c)
    
    result = []
    pass_thres = False
    for i in range(len(temp)):
        if temp[-(i+1)] >= n:
            pass_thres = True
            result.append(True)
        elif temp[-(i+1)] != 0 and pass_thres:
            result.append(True)
        else:
            pass_thres = False
            result.append(False)
    result = result[::-1]

    return pd.Series(result)

In [5]:
# function which returns a filtered version of a given dataframe which only includes the entries which are part of a consecutive series of length n Trues for a boolean mask
def consec_filter(frame, n):
    threshold_mask = (frame['Temp'] < frame['lower']) | (frame['Temp'] < frame['static'])
    
    contig_mask = pd.DataFrame(data=consec_count_n(threshold_mask, n))

    contig_mask.reset_index()
    contig_mask.columns = ["contiguous"]
    contig_mask.index = frame.index

    filtered_data = frame[frame.index.isin(contig_mask.index[contig_mask['contiguous']])]

    return filtered_data

In [6]:
# function to turn the dataframe of days into flat data
def flatten(frame):
    sequences = []

    # loop to find and append the sequences and their lengths to a new array
    for zip_code, group in frame.groupby(level='GEOID20'):
        start_date = None
        sequence_length = 0
        prev_date = None
        for date in group.index.get_level_values('Date'):
            if start_date is None:
                start_date = date
                sequence_length = 1
            elif prev_date is not None and (date - prev_date).days == 1:
                sequence_length += 1
            else:
                sequences.append({
                    'GEOID20': zip_code,
                    'Start_Date': start_date.date(),
                    'Sequence_Length': sequence_length
                })
                start_date = date
                sequence_length = 1
            prev_date = date
        if sequence_length > 0:
            sequences.append({
                'GEOID20': zip_code,
                'Start_Date': start_date.date(),
                'Sequence_Length': sequence_length
            })
    # flatten to 2d array
    result = [[seq['GEOID20'], seq['Start_Date'], seq['Sequence_Length']] for seq in sequences]

    return result

In [7]:
# function to do the whole process for a single year
def find_coldspells(frame, cutoff, days, year):
    processed = proc(frame, cutoff, year)
    days = consec_filter(processed, days)
    result = flatten(days)
    return result

In [8]:
def mCounts(dfs, thresholds):
    # Assuming you have a list of dataframes named dfs, where each dataframe corresponds to a year
    # Concatenate the dataframes into one dataframe
    combined_df = pd.concat(dfs)

    # Extract year and month from datetime column
    combined_df['Year'] = combined_df['datetime'].dt.year
    combined_df['Month'] = combined_df['datetime'].dt.month

    # Initialize a dictionary to store counts for each threshold
    threshold_counts = {}

    # Iterate over each threshold
    for threshold in thresholds:
        # Filter the dataframe to get only entries exceeding the threshold
        exceed_threshold_df = combined_df[combined_df['threshold_column'] > threshold]
        
        # Group by zip code, year, and month and count the number of exceedances
        grouped_df = exceed_threshold_df.groupby(['GEOID20', 'Year', 'Month']).size().reset_index(name=f'Exceedances_{threshold}')
        
        # Store the counts for this threshold in the dictionary
        threshold_counts[threshold] = grouped_df

        # Merge the dataframes for each threshold on zip code, year, and month
        result_df = threshold_counts[thresholds[0]]
        for threshold in thresholds[1:]:
            result_df = pd.merge(result_df, threshold_counts[threshold], on=['GEOID20', 'Year', 'Month'], how='outer').fillna(0)

    # Now result_df contains the desired format with zip code, year, month, and the count of exceedances
    return result_df

In [22]:
columnNames = ['Zip_Code', 'Starting_Date', 'Days']

years = range(2006, 2023)

for year in years:
    frame = pd.read_csv("extracted_by_year/" + str(year) + "_extracted/tmin" + str(year) + ".csv")
    result = find_coldspells(frame, CUTOFF, DAYS, year)
    np.savetxt("coldspell_" + str(year)  + "_cutoff" + str(CUTOFF) +  ".csv", result, fmt='%s', delimiter=', ', header=', '.join(columnNames))

: 

In [12]:
# find for the various threholds the number of days per month, all in one file
thresholds = [-6.67, -1.11, 4.44]
years = range(2006, 2008)

yFrames = []

for year in years:
    frame = pd.read_csv("extracted_by_year/" + str(year) + "_extracted/tmin" + str(year) + ".csv")
    filter = 
    yFrames.append(frame)

mList = mCounts(yFrames, thresholds)
mList.head()

KeyError: 'datetime'